In [ ]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import os
import implicit

In [ ]:
data_path = 'data/'                         #Загружаем данные
data_1 = 'data_learn_match.csv'
data_2 = 'data_test_match.csv'
data_3 = 'data_target_thread.csv'
raw_data = pd.read_csv(os.path.join(data_path, data_1), header=None)
raw_data.drop(raw_data.columns[2:4], axis=1, inplace=True)
raw_data['values'] = 1
raw_data

In [ ]:
raw_data.columns = ['user_uid', 'thread_uid', 'values']
data = raw_data.copy()

In [ ]:
list_match_user = data['user_uid'].unique()  #создаем список уникальных пользователей

In [ ]:
list_match_thread = data['thread_uid'].unique()  #создаем список уникальных складчин

In [ ]:
raw_data_2 = pd.read_csv(os.path.join(data_path, data_2), header=None)
raw_data_2['values'] = 1

In [ ]:
raw_data_2.columns = ['user_uid_test', 'thread_uid_test', 'values']
data_2 = raw_data_2.copy()
data_2

In [ ]:
list_test_user = data_2['user_uid_test'].unique()  #аналогично, только на тестовом наборе

In [ ]:
list_test_thread = data_2['thread_uid_test'].unique()

In [ ]:
data_threads_test = pd.read_csv(os.path.join(data_path, data_3), header=None)
data_threads_test.columns = ['thread_uid_test']
data_threads_test

In [ ]:
data_threads_test = data_threads_test['thread_uid_test']

In [ ]:
data_threads_filter = list(set(list_match_thread).difference(data_threads_test)) #список для отсева неактуальных складчин

In [ ]:
list_test_user.sort()

In [ ]:
sparse_item_user = sparse.csr_matrix((data['values'].astype(float), (data['thread_uid'], data['user_uid'])))
sparse_user_item = sparse.csr_matrix((data['values'].astype(float), (data['user_uid'], data['thread_uid'])))

In [ ]:
sparse_user_item.shape

In [ ]:
#СОбственно, сама модель и ее обучение. 
model = implicit.als.AlternatingLeastSquares(factors=2000, regularization=0.01, iterations=30)
alpha_val = 4
data_conf = (sparse_item_user*alpha_val).astype('double')
model.fit(data_conf)

In [ ]:
#Рекомендации. Для каждого пользователя предсказываем 50 складчин (отфильтровав старые), затем фильтруем результаты по медиане.
rec = []
scores = []
for userid in list_test_user:
    recommended = model.recommend(userid, sparse_user_item, N=50, filter_already_liked_items = True, filter_items = data_threads_filter)
    for score in recommended:
        scores.append(score[1])
    median = np.median(scores)
    condition = lambda x: x[1] >= median
    filtered_recommended = list(filter(condition, recommended))
    recommended_1 = [x[0] for x in filtered_recommended]
    rec.append(recommended_1)
rec

Далее собственно обработка результатов, подсчет метрик. Для каждого юзера создается список складчин из тестового набора и список из предсказанных. Пересечение этих списков деленое на длину списка предсказаний = precision, деленое на длину списка из тестового набора = recall.

In [ ]:
data_2['columns'] = data_2.groupby('user_uid_test')['thread_uid_test'].cumcount()

In [ ]:
result = data_2.pivot(index='user_uid_test', columns='columns')

In [ ]:
result[0:3]

In [ ]:
result.drop(['values'], axis=1, inplace=True)

In [ ]:
result['concat_col']=result.apply(lambda row: row.dropna().tolist(), axis=1)

In [ ]:
result['concat_col']

In [ ]:
df_test_threads = result[['concat_col']]
df_test_threads

In [ ]:
df_test_threads['predict'] = rec

In [ ]:
df_test_threads

In [ ]:
relev = []
for x in range(df_test_threads.shape[0]):
    A = list(set(df_test_threads['concat_col'].iloc[x]) & set(df_test_threads['predict'].iloc[x]))
    relev.append(len(A))

In [ ]:
df_test_threads['relevant'] = relev

In [ ]:
df_test_threads

In [ ]:
df_test_threads['len_concat'] = df_test_threads['concat_col'].apply(lambda x: len(x))
df_test_threads['len_predict'] = df_test_threads['predict'].apply(lambda x: len(x))

In [ ]:
df_test_threads['precision'] = df_test_threads['relevant']/df_test_threads['len_predict']
df_test_threads['recall'] = df_test_threads['relevant']/df_test_threads['len_concat']

In [ ]:
list_drop = list(set(list_test_user).difference(list_match_user))

In [ ]:
list_drop

In [ ]:
df_test_threads_f = df_test_threads.drop(list_drop, axis = 0)
df_test_threads_f[0:15]

In [ ]:
Precision = df_test_threads_f[['precision']].mean()
Precision

In [ ]:
Recall = df_test_threads_f[['recall']].mean()
Recall